#### Retrieval Augemnated Generation 


In [3]:
from langchain_community.document_loaders import UnstructuredURLLoader

url = ['https://www.welt.de/wirtschaft/article254899998/Deutsche-Bahn-Verkehrsminister-Wissing-warnt-vor-der-Zerschlagung.html',
       'https://de.wikipedia.org/wiki/Deutsche_Bahn'
       ]

loader = UnstructuredURLLoader(urls=url)
data = loader.load()

In [4]:
len(data)

## warum die len ist 2 : wegens des, hamma 2 url dabei

2

In [5]:
data

[Document(metadata={'source': 'https://www.welt.de/wirtschaft/article254899998/Deutsche-Bahn-Verkehrsminister-Wissing-warnt-vor-der-Zerschlagung.html'}, page_content='Wirtschaft\n\n0\n\nVerkehrsminister\n\n„Null Effekt auf die Pünktlichkeit der Züge“ – Wissing warnt vor Zerschlagung der Bahn\n\nVon Philipp VetterWirtschaftskorrespondent\n\nStand: 18.12.2024Lesedauer: 3 Minuten\n\nIm letzten Wahlkampf forderte die FDP, den Betrieb der Züge und des Schienennetzes der Bahn zu trennen. Auch die Union forciert in ihrem aktuellen Programm eine stärkere Trennung. Doch nun warnt Ex-FDP-Verkehrsminister Wissing vor der Idee. Vor allem eine Sorge treibt ihn um.\n\n0\n\nAnzeige\n\nEs ist erst drei Jahre her, dass Volker Wissing, der inzwischen parteilose Verkehrsminister, den Koalitionsvertrag der Ampel-Regierung mit ausgehandelt hat. Damals hatten sowohl Wissings frühere Partei, die FDP, als auch einige Grüne mit der Idee geliebäugelt, die Deutsche Bahn aufzuspalten.\n\nZwei getrennte Unternehme

so now to divide our data into smaller , why do we need to divide our data into chunks ?
- helps maintain the context of the text
- reducing computational and improving overall performance
- allows for more efficient indexing and retrieval of relevant information
- better overall summaries

and why it is useful for doing chunking ? because when user will ask questions, it will only give that related answer by searching around

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#  tiktoken library for more precise token counting when splitting text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,  # This will now be in tokens, not characters
    chunk_overlap=0
)
docs = text_splitter.split_documents(data)

# total number of splitted documents 
print("Total number of documents:", len(docs))


Total number of documents: 125


In [7]:
docs[5]
# dokument, das sich auf Index [5] befindet

Document(metadata={'source': 'https://www.welt.de/wirtschaft/article254899998/Deutsche-Bahn-Verkehrsminister-Wissing-warnt-vor-der-Zerschlagung.html'}, page_content='Sie sollen voraussichtlich auch nicht mehr in dieser Legislaturperiode freigegeben werden. Das verunsichere die Baubranche, die ihre Kapazitäten aufgrund der unklaren Lage nicht erhöhten, das führe dann wiederum zu höheren Kosten, warnt Wissing.\n\n„Wichtig ist, dass der Haushaltsgesetzgeber jetzt auch den Weg freimacht für die Finanzierung dieses Konzepts“, sagt der Minister. „Jede Form von Sperren im Haushalt, jede Form von Verzögerungen bei der Freigabe von Mitteln führt zu einem erheblichen Mehraufwand und zu Verunsicherung bei der Bahn, aber auch bei der Bauindustrie.“')

Jetzt fange mit Embeddings an. warum brauchen wir das ?
- because we want to convert our text data into numerical representation 

In [8]:
# # embedding models 
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
docs_text = [doc.page_content for doc in docs]  # Extract text
embeddings = model.encode(docs_text)


## only to check whether the above is working on not 
vector = model.encode("Hello, Servus") # einfach beispiel 
vector[:5]

/Users/killuaa/Desktop/LLM_porjects/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([-0.01166679,  0.05221787,  0.06761136,  0.01808612, -0.04677953],
      dtype=float32)

In [9]:
import faiss
import numpy as np

embeddings_array = np.array(embeddings, dtype="float32")

# Step 1: Initialize FAISS index
embedding_dim = embeddings_array.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(embedding_dim)   # L2 (Euclidean) distance index

# Step 2: Add embeddings to the index
index.add(embeddings_array)
print(f"Number of embeddings added to the index: {index.ntotal}")


Number of embeddings added to the index: 125


In [10]:
# Step 1: Generate embedding for the query
query_vector = model.encode("Nürnberg").astype("float32")

# Step 2: Perform the search
k = 3 # Number of nearest neighbors to retrieve
distances, indices = index.search(np.array([query_vector]), k)

# Step 3: Retrieve corresponding documents
print("Query Results:")
for i, idx in enumerate(indices[0]):
    print(f"Result {i + 1}: {docs_text[idx]} (distance: {distances[0][i]:.4f})")

Query Results:
Result 1: trat ein neuer Fahrplan in Kraft. Die Intercity-Express-Linien München–Nürnberg–Leipzig–Berlin–Hamburg und München–Nürnberg–Frankfurt–Köln–… verkehren seitdem im Stundentakt. Zwischen Nürnberg und München sowie zwischen Frankfurt und Köln wurde ein Intercity-Express-Halbstundentakt hergestellt, in Leipzig wurde ein stündlicher Intercity-Express-Anschluss von und nach Dresden geschaffen. Durch die Vollinbetriebnahme der Schnellfahrstrecke Nürnberg–Ingolstadt reduzierten sich viele Reisezeiten von und nach München um rund eine halbe Stunde. Auf der Franken-Sachsen-Magistrale lösten Regionalzüge die vormals eingesetzten Intercitys ab. (distance: 1.3004)
Result 2: Vorstände: Richard Lutz (Vorsitzender) | Berthold Huber (Infrastruktur) | Levin Holle (Finanzen und Logistik) | Evelyn Palla (Regionalverkehr) | Sigrid Nikutta (Güterverkehr) | Michael Peterson (Personenfernverkehr) | Daniela Gerd tom Markotten (Digitalisierung und Technik) | Martin Seiler (Personal und R

In [11]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain_core.output_parsers import StrOutputParser
import torch 


In [12]:
model_id = "distilbert/distilgpt2"

In [13]:
text_generation_pipeline = pipeline(
    "text-generation",model=model_id,torch_dtype=torch.float32 , max_new_tokens=50, device=0)

Device set to use mps:0
